# FAST-UAV - Multirotor Design Optimization with Off-The-Shelf components
*Author: Félix Pollet - 2023* <br>

In this notebook, we will see how to achieve a design optimization using real off-the-shelf components instead of continuous estimation models.

In fact, the default use of FAST-UAV is based on design models that parameterize the components in the continuous domain. However, in some situations the designer may want to restrict the design space to obtain the optimal UAV whose components are available on the market.

For this reason, FAST-UAV has the ability to optimize the design of a UAV using catalogues of existing components. Mixed designs, i.e. combining 'custom' components defined with continuous estimation models and 'off-the-shelf' components, are also possible.

The user is referred to the following publication for further details:
> F. Pollet, M. Budinger, S. Delbecq, J. -M. Moschetta, and J. Liscouët. Quantifying and Mitigating Uncertainties in Design Optimization Including Off-the-Shelf Components: Application to an Electric Multirotor UAV. Aerospace Science and Technology, 2023, pp.108179. https://doi.org/10.1016/j.ast.2023.108179.

In the present notebook, we re-use the multirotor UAV example from the [Tutorial](0_Tutorial.ipynb) and [1_Multirotor_Design](1_Multirotor_Design.ipynb) notebooks, but the propeller selection in the optimization process is restricted to a set of components from the manufacturer APC, which is provided [here](../data/catalogues/Propeller/APC_propellers_MR.csv).

## 1. Setting up a problem with off-the-shelf components

In [ ]:
# Import required librairies
import os.path as pth
import sys
import logging
import fastoad.api as oad
import shutil
from fastuav.utils.postprocessing.analysis_and_plots import mass_breakdown_sun_plot_drone, mass_breakdown_bar_plot_drone, multirotor_geometry_plot
import cma
from fastuav.utils.drivers.cmaes_driver import CMAESDriver
from time import time

sys.path.append(pth.abspath("."))

#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.display import display, HTML, IFrame
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# Declare paths to folders and files
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_mdo_cots.yaml")
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_inputs_DJI_M600.xml")  # You may also provide the output file from the continuous optimization problem to get a starting point for the new optimization.
# SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_outputs_DJI_M600_mdo.xml")

The changes in the model rely in the use of catalogues instead of continuous estimation models to select the components. This is described in the configuration file by the `off_the_shelf` option when defining the model:

```yaml
model:
    scenarios:
        id: fastuav.scenarios.multirotor
    propulsion:
        id: fastuav.propulsion.multirotor
        off_the_shelf_propeller: True
        off_the_shelf_motor: False
        # ...
```

By default, these options are set to False. 

You can check with the N2 diagram visualization that the `catalogue_selection` module has replaced the `skip_catalogue_selection` module for components where the option has been set to True. In a nutshell, the design variables varied by the optimizer are now mapped to the closest component available in the catalogue.

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

In [ ]:
oad.variable_viewer(input_file)

## 2. Multirotor MDO with off-the-shelf components

Due to the non-continuous nature of the selection process, the use of a gradient-based algorithm is risky. We therefore recommend the use of an evolutionary algorithm, [CMA-ES](http://www.cmap.polytechnique.fr/~nikolaus.hansen/cmaesintro.html).

### a) Using a gradient-based algorithm: SLSQP
*Optimization with SLSQP is likely to fail due to the non-continuous, rugged nature of the problem with off-the-shelf components.*

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

### b) Using an evolutionnary strategy: CMA-ES
The [pycma](https://github.com/CMA-ES/pycma) implementation of CMA-ES is used. The optimizer is wrapped with a modified version of the [OpenMDAO driver](https://github.com/OpenMDAO/RevHack2020/tree/master/problems/cma_es).<br>
The optimization takes longer than the 'ideal' one performed with continuous estimation models and a gradient-based algorithm. In the meantime you can take a break (but not too long, the convergence is of the order of a few minutes!). Also, you may have to re-run the optimization a few times to make sure a feasible and optimal solution was found.

*Note that the use of custom-made drivers is not supported yet by FAST-UAV. Consequently, the setup and run of the problem is made by hand.*

In [ ]:
# Get problem configuration
conf = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
prob = conf.get_problem(read_inputs=True, auto_scaling=True)

# CMA-ES Driver setup (see cma.CMAOptions() for all possible options for CMA-ES)
driver = prob.driver = CMAESDriver()
driver.options["sigma0"] = 0.1
driver.options[
    "penalty_parameter"
] = 1.0e8  # should be chosen such that f and the constraints violation have a similar magnitude.
driver.options["penalty_exponent"] = 1.0
driver.CMAOptions["tolfun"] = 1e-6
driver.CMAOptions["tolfunhist"] = 1e-5
driver.CMAOptions["popsize_factor"] = 1
# driver.options["restarts"] = 1
# driver.options["restart_from_best"] = True
# driver.CMAOptions['CMA_stds'] = [1, 1, 100, 0.1, 0.1, 1, 1, 1, 1]
# driver.options['augmented_lagrangian'] = True  # whether to use penalty method or augmented lagrangian for handling constraints

# Setup and run problem
prob.setup()
start_time = time()
prob.optim_failed = prob.run_driver()  # optimize
end_time = time()
computation_time = round(end_time - start_time, 2)
prob.write_outputs()

print("Objective function value ", driver.get_objective_values())

logger = cma.plot()  # plot logger

Let's save and visualize the optimization results:

In [ ]:
output_file = optim_problem.output_file_path
shutil.copy(output_file, 
            pth.join(SOURCE_FOLDER_PATH, 'problem_outputs_discrete_DJI_M600_mdo.xml')
)

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

In [ ]:
oad.variable_viewer(output_file)

## 3. Analysis and plots

We may now compare the results from the optimization in the continuous domain with the off-the-shelf (or mixed) configuration:

In [ ]:
OUTPUT_FILE_CONTINUOUS = pth.join(SOURCE_FOLDER_PATH, "problem_outputs_DJI_M600_mdo.xml")
OUTPUT_FILE_DISCRETE = pth.join(SOURCE_FOLDER_PATH, "problem_outputs_discrete_DJI_M600_mdo.xml")
fig = multirotor_geometry_plot(OUTPUT_FILE_CONTINUOUS, name="Drone custom")
fig = multirotor_geometry_plot(OUTPUT_FILE_DISCRETE, name="Drone off-the-shelf", fig=fig)
fig.show()

In [ ]:
fig = mass_breakdown_sun_plot_drone(OUTPUT_FILE_DISCRETE)
fig.show()

In [ ]:
fig = mass_breakdown_bar_plot_drone(OUTPUT_FILE_CONTINUOUS, name="Custom design")
fig = mass_breakdown_bar_plot_drone(OUTPUT_FILE_DISCRETE, name="Off-the-shelf design", fig=fig)
fig.show()